In [ ]:
! ls /Users/guillermogonzalez/ga/evan_github/input/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Train Data EDA

In [2]:
df_train = pd.read_csv('../input/train.csv')

In [ ]:
columns = df_train.columns
for col in columns:
    print(col)
    print(df_train[col].value_counts().sort_values(ascending=False))

In [ ]:
df_train['Date'].nunique()

In [ ]:
df_train.nunique()

In [3]:
train = df_train.drop(labels=['Address','Block','Street',
                            'AddressNumberAndStreet',
                           'AddressAccuracy'],axis=1)

In [4]:
train['Date'] = pd.to_datetime(train['Date']) # convert dtype to date

In [ ]:
train.info()

In [ ]:
plt.hist(x=train['WnvPresent'])
plt.show()

In [ ]:
plt.hist(x=train['NumMosquitos'])
plt.show()

In [ ]:
train['Species'].sort_values()

In [ ]:
plt.hist(x=train['Species'].sort_values(),orientation='vertical')
plt.xticks(x=train['Species'], rotation='vertical')
plt.title('Species Histogram')
plt.xlabel('Species',size='large')
plt.ylabel('Counts',size='large')
plt.show()

### Test Data EDA

In [5]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
columns_test = df_test.columns

for col in columns_test:
    print(col)
    print(df_test[col].value_counts())

In [6]:
df_test[ df_test['Trap']=='T094B' ]

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
1192,1193,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1193,1194,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX RESTUANS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1194,1195,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX PIPIENS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1195,1196,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX SALINARIUS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1196,1197,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX TERRITANS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1197,1198,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX TARSALIS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1198,1199,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",UNSPECIFIED CULEX,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
1199,1200,2008-06-11,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX ERRATICUS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
2400,2401,2008-06-17,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9
2401,2402,2008-06-17,"9600 South Longwood Drive, Chicago, IL 60643, USA",CULEX RESTUANS,96,S LONGWOOD DR,T094B,"9600 S LONGWOOD DR, Chicago, IL",41.71914,-87.669539,9


In [ ]:
df_test['Trap'].nunique()

In [ ]:
df_test['Date'].nunique()

In [ ]:
df_test['Id'].nunique()

In [7]:
test = df_test.drop(labels=['Address','Block','Street',
                            'AddressNumberAndStreet',
                           'AddressAccuracy'],axis=1)

In [ ]:
test['Date'] = pd.to_datetime(test['Date']) # convert date dtype to date

In [ ]:
test.info()

In [ ]:
plt.hist(x=test['Species'].sort_values(),orientation='vertical')
plt.xticks(x=test['Species'], rotation='vertical')
plt.title('Species Histogram')
plt.xlabel('Species',size='large')
plt.ylabel('Counts',size='large')
plt.show()

### Weather data EDA

In [8]:
used_columns = ['Station',
                'Date',
                'Tmax',
                'Tmin',
                'Tavg',
                'Depart',
                'DewPoint',
                'WetBulb',
                'Heat',
                'Cool',
                'Sunrise',
                'Sunset',
                'CodeSum',
                #'Depth',
                #'SnowFall',
                'PrecipTotal',
                'SeaLevel',
                'ResultSpeed',
                'ResultDir',
                'AvgSpeed']

In [17]:
weather = pd.read_csv('../input/weather.csv',
                      usecols=used_columns,
                      na_values={'Tavg':'M',
                                 'WetBulb':'M',
                                 'Heat':'M',
                                 'Cool':'M',
                                 'Sunrise':'-',
                                 'Sunset':'-',
                                 'PercipTotal':'M'
                                }
                     )

In [18]:
# converting weather codes to one-hot encoded columns
code_list = list(set([i[j] for i in [title.split() for title in pd.get_dummies(weather.CodeSum).columns.tolist()] for j in range(len(i))]))

for code in code_list:
    weather[code] = [1 if code in weather.CodeSum[i] else 0 for i in range(weather.CodeSum.shape[0])]

In [19]:
weather

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,GR,FG+,SN,FU,VCFG,MIFG,DZ,HZ,VCTS,SQ
0,1,2007-05-01,83,50,67.0,14,51,56.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68.0,M,51,57.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51.0,-3,42,47.0,14.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-02,60,43,52.0,M,42,47.0,13.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,1,2007-05-03,66,46,56.0,2,40,48.0,9.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,2,2007-05-03,67,48,58.0,M,40,50.0,7.0,0.0,...,0,0,0,0,0,0,0,1,0,0
6,1,2007-05-04,66,49,58.0,4,41,50.0,7.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,2,2007-05-04,78,51,NaN,M,42,50.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
8,1,2007-05-05,66,53,60.0,5,38,49.0,5.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,2,2007-05-05,66,54,60.0,M,39,50.0,5.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
weather['Date'] = pd.to_datetime(weather['Date']) # convert date dtype to date

def median_fill(df,columns):
    '''
    takes list of column names, changes data type to float and converts null values to the median value.
    '''
    for c in columns:
        try:
            df[c] = df[c].copy().astype(dtype='float') # converts to float
        except:
            print(c,df[c].dtype)
        df[c][df[c].isnull() == True] = df[c][df[c].isnull() == True].copy().apply(lambda x: df[c].median())
    #return df[c][df[c].isnull() == True].value_counts

median_fill(weather,['Tavg','WetBulb','Heat','Cool']) # convert nulls to median value
# converts "Trace" percipitation to very small non-zero float
#weather['PercipTotal'][weather['PercipTotal'] == '  T'] = weather['PercipTotal'][weather['PercipTotal'] == '  T'].copy().apply(lambda x: 0.001)
#median_fill(weather,['PercipTotal']) # convert nulls to median value

/anaconda2/envs/ga/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [ ]:
# Farenheit to Celcius for RH calculation
weather[['Tavg(C)','DewPoint(C)']]=weather[['Tavg','DewPoint']].apply(lambda x: (x-32)*(5/9)).round(1)

In [ ]:
weather.head()

In [ ]:
weather.info()

In [ ]:
#Calculation Relative Humidity
weather['RH'] = (100*(1.8096 + np.exp(17.2694*weather['DewPoint(C)']/(237.3 + weather['DewPoint(C)'])))/
                 (1.8096 + np.exp((17.2694*weather['Tavg(C)']/(237.3 + weather['Tavg(C)']))))).round(1)

In [ ]:
weather.shape

In [ ]:
weather.groupby('Date')[['Tmax','Tmin','Tavg','DewPoint','WetBulb','Heat','Cool']].mean().shape

### Spray Data EDA

In [ ]:
df_spray = pd.read_csv('/Users/guillermogonzalez/ga/evan_github/input/spray.csv')

In [ ]:
df_spray.nunique()

In [ ]:
columns_spray = df_spray.columns
for col in columns_spray:
    print(col)
    print(df_spray[col].value_counts())

In [ ]:
df_spray.isnull().sum()

In [ ]:
spray = df_spray.copy()

In [ ]:
spray['Time']

### Sample Submission data EDA

In [ ]:
df_sample = pd.read_csv('/Users/guillermogonzalez/ga/evan_github/input/sampleSubmission.csv')

In [ ]:
df_sample.nunique()

### Date Ranges

In [ ]:
print('          Min          Max')
print('train =  ',df_train['Date'].min(),' ', df_train['Date'].max())
print('test =   ',df_test['Date'].min(),' ', df_test['Date'].max())
print('weather =',df_weather['Date'].min(),' ', df_weather['Date'].max())
print('spray =  ',df_spray['Date'].min(),' ', df_spray['Date'].max())